In [1]:
# установить свежую версию
!pip install scipy==1.5.2

     |████████████████████████████████| 25.9MB 149kB/s 
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.2 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [2]:
import math
import numpy as np
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt 

Постройте линейную регрессионную модель зависимости веса новорожденного от расы и образования матери, срока беременности, а также информации о курении матери во время беременности.

Формат файла: столбец 1: раса (1 – негроидная, 0 - другая), столбец 2: образование (от 0 до 16), столбец 3: курила ли (1 да, 0 нет), столбец 4: срок беременности (в неделях), столбец 5: вес новорожденного (в граммах) 

Пусть фиктивные переменные для расы и для курения принимают те же значения, что и сами переменные.

*  Найдите оценку параметра $\beta_0$.

*  Найдите оценку параметра при переменной раса.

*  Найдите оценку дисперсии ошибок наблюдений.

*  Найдите точность 95% доверительного интервала для параметра при сроке беременности.

*  Найдите коэффициент детерминации.

* Проверьте гипотезу о значимости всей модели в целом. Найдите p-значение и примите статистическое решение о значимости модели при уровне значимости 0.05

* Какие параметры модели значимы?

* Чему равен коэффициент Акаике построенной модели?

* Рассмотрите модель зависимости веса новорожденного от расы и срока беременности. Чему равен коэффициент Акаике в данной модели?

In [45]:
df = pd.read_csv('weights.txt', sep='\t')
df.head(2)

,race,educ,smoke,preg,weight
0,1,8,0,39,3670
1,1,9,1,38,2775


In [46]:
y = df.weight.values.reshape(-1, 1)
columbs = ['race', 'educ', 'preg', 'smoke']
A = df[columbs].values

In [47]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(y=y, X=A)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Найдите оценку параметра $\beta_0$
​	

In [48]:
lr.intercept_

array([-2841.37767615])

Найдите оценку параметра при переменной раса, у нас это коэф. $\beta_1$

In [49]:
lr.coef_

array([[-211.57152451,    7.24940335,  157.75158661,  -44.25278218]])

Найдите оценку дисперсии ошибок наблюдений

$S^2 = \frac{RSS}{n-k-1}$где

$k$ число параметров модели

$n$ объем выборки

$RSS = \sum \limits_{i=1}^n e_i^2 = \sum \limits_{i=1}^n (y - \hat{y})^2$

In [50]:
k = 4
n = df.smoke.count()

y_pred = lr.predict(A)

RSS = np.sum((y - y_pred)**2)

S_square = RSS / (n - k - 1)

S = np.sqrt(S_square)

S_square

193390.286117486

Найдите точность 95% доверительного интервала для параметра при сроке беременности

Параметр при сроке беременности $\beta_3$

$\beta_j$

$\hat{\beta_j} \pm t_{1 - \frac{\alpha}{2}, n-k-1} S \sqrt{[(A^TA)^{-1}]_{(j+1)(j+1)}}$

$[(A^TA)^{-1}]_{(j+1)(j+1)}$ такая запись, говорит нам, что необходимо взять из $(j+1)$ строки и $(j+1)$ столбца элемент из матрицы.

например, нам нужно взять элемент из матрицы $[(A^TA)^{-1}]_{(j+1)(j+1)}$ участвующий при построении оценки $\hat{\beta_{j=2}}$, здесь $j=2$, значит нам нужно взять из $2+1$ строки и $2+1$ столбца элемент. 

Точность

$t_{1 - \frac{\alpha}{2}, n-k-1} S \sqrt{[(A^TA)^{-1}]_{(j+1)(j+1)}}$

Тонкий момент, в данном случае нам нужно для вычисления элемента $[(A^TA)^{-1}]_{(2+1)(2+1)}$ использовать матрицу $A$, где первым столбцом будет столбец из единиц. Поэтому нужно будет перетренировать с такой же матрицей линейный регрессор. Иначе в матрице получаются не верные значения.

In [77]:
# наблюдаемые значения, отклики
y = df.weight.values.reshape(-1, 1)

# матрица факторов с первм столбцом из единиц
n = df.smoke.count()
columbs = ['race', 'educ', 'preg', 'smoke']
A = np.hstack((np.ones(n).reshape(-1, 1), df[columbs].values))

# модель
lr = LinearRegression()
lr.fit(y=y, X=A)

# предсказания
y_pred = lr.predict(A)

# k - кол-во факторов, без учёта столбца с единицами
k = A.shape[-1] - 1

RSS = np.sum((y - y_pred)**2)
# оценка дисперсии
S_square = RSS / (n - k - 1)
# оценка среднеквадратичного отклонения
S = np.sqrt(S_square)

In [78]:
# квантиль
alpha = 0.05
t = st.t.ppf(1 - (alpha/2), df=(n-k-1))

In [79]:
AA_inved = np.linalg.inv(A.T @ A)

accuracy = t * S * np.sqrt(AA_inved[3][3])
accuracy

55.38424956665776

Найдите коэффициент детерминации

Коэффициент детерминации

$R^2 = K^2_{\hat{y}y} = \frac{ \bigg ( \sum \limits_{i=1}^n (y_i - \overline Y)(\hat{y_i} - \overline Y) \bigg )^2 }{\sum \limits_{i=1}^n (y_i - \overline Y)^2 \sum \limits_{i=1}^n (\hat{y_i} - \overline Y)^2} = 1 - \frac{RSS}{\sum \limits_{i=1}^n(y_i - \overline Y)^2}$

In [63]:
R_square = 1 - (RSS / np.sum( (y - y.mean())**2 ) )
R_square

0.33646521179872235

Проверьте гипотезу о значимости всей модели в целом.

Найдите p-значение и примите статистическое решение о значимости модели при уровне значимости 0.05

Гипотеза о значимости всей модели

$H_0: \beta_{k_1} = ... = \beta_{k_q} = 0, k_i \neq 0$

$H_1: \exists \beta_{k_i} \neq 0$



Статистический критерий

$F = \frac{R^2}{1 - R^2} \frac{n-k-1}{k} \sim F_{k, n-k-1}$, где

$F_{k, n-k-1}$ распределение Фишера.

Как найти $p_{value}$?

По построению $V_k$ у нас правосторонняя. 

$V_k = (F_{1 - \alpha, k, n-k-1}, + \infty)$

$F_{1 - \alpha, k, n-k-1}$ квантиль фишера

Тогда $p_{value}$ будем искать по формуле [отсюда](https://stepik.org/lesson/26216/step/3?unit=8140)

$p_{value} = 1 - F_{1 - \alpha, k, n-k-1}(F_в | H_0)$ при условии, что $H_0$ верна.

In [85]:
Fv = (R_square / (1 - R_square)) * ((n-k-1) / k )
Fv

12.04314969209367

In [87]:
p_value = st.f.sf(Fv, k, n-k-1)
p_value

5.868362739122455e-08

Какие параметры модели значимы?

$H_0: \beta_j = 0$

$H_1: \beta_j \neq 0$

$t_{\beta_j} = \frac{\hat{\beta_j}}{S \sqrt{[(A^TA)]^{-1}_{(j+1)(j+1)}}} \sim T_{n-k-1}$

$T_{n-k-1}$ распр. Стьюдента.

$[(A^TA)^{-1}]_{(j+1)(j+1)}$ такая запись, говорит нам, что необходимо взять из $(j+1)$ строки и $(j+1)$ столбца элемент из матрицы.

$V_k = (t_{1 - \frac{\alpha}{2}, n-k-1}, + \infty)$

$t_{1 - \frac{\alpha}{2}}$ квантиль распр. Стьюдента.

$|t_{\beta_{j} выб}| \in V_k \Rightarrow H_0$ отклоняется.

In [163]:
def hypothesis_testing(A, alpha=0.05):
  num_betas = A.shape[-1] - 1
  for j in range(1, num_betas + 1):
    beta_j = lr.coef_[0][j]
    tbeta = beta_j / (S * np.sqrt(AA_inved[j][j]))
    p_value = st.t.sf(abs(tbeta), n-k-1)
    print(f'{df[columbs].columns[j-1]} значим' if p_value < alpha else f'{df[columbs].columns[j-1]} не значим')

In [164]:
hypothesis_testing(A)

race значим
educ не значим
preg значим
smoke не значим


Чему равен коэффициент Акаике построенной модели?

$AIC = 2k + n \cdot (ln(\frac{RSS}{n}) + 1)$

In [171]:
AIC = (2 * k) + (n * (np.log(RSS/n) + 1))
AIC

1320.1172339452087

Рассмотрите модель зависимости веса новорожденного от расы и срока беременности. Чему равен коэффициент Акаике в данной модели?

In [172]:
# наблюдаемые значения, отклики
y = df.weight.values.reshape(-1, 1)

# матрица факторов с первм столбцом из единиц
n = df.smoke.count()
columbs = ['race', 'preg']
A = np.hstack((np.ones(n).reshape(-1, 1), df[columbs].values))

# модель
lr = LinearRegression()
lr.fit(y=y, X=A)

# предсказания
y_pred = lr.predict(A)

# k - кол-во факторов, без учёта столбца с единицами
k = A.shape[-1] - 1

RSS = np.sum((y - y_pred)**2)

AIC = (2 * k) + (n * (np.log(RSS/n) + 1))
AIC

1316.6439727031286